In [1]:
import os
TS_data_path = '/home/ec2-user/SageMaker/data/Toronto_Star_Publication_with_query/'
GM_data_path = '/home/ec2-user/SageMaker/data/The_Globe_and_Mail_with_DP_filter_by_article_type/'

filenames = []

for filename in os.listdir(TS_data_path):
    filenames.append(TS_data_path+filename)
    
for filename in os.listdir(GM_data_path):
    filenames.append(GM_data_path+filename)
    
print(len(filenames))

192433


In [2]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['textcat','ner','lemmatizer','parser','tagger'])
stopwords = nlp.Defaults.stop_words

In [3]:
from lxml import etree
import tqdm
from bs4 import BeautifulSoup
import datetime

def get_content(filename):
    tree = etree.parse(filename)
    root = tree.getroot()
    if root.find('.//HiddenText') is not None:
        text = (root.find('.//HiddenText').text)

    elif root.find('.//Text') is not None:
        text = (root.find('.//Text').text)

    else:
        text = None
    if text is None:
        return ''
    text = BeautifulSoup(text, parser='html.parser').get_text()
    title = root.find('.//Title').text
    return f'{title}. {text}'

from collections import defaultdict
writer = open('vocab/vocab.txt','w')
freq = defaultdict(int)

for idx,filename in enumerate(filenames):
    if (idx) % 10000==0:
        print(f'Processed {idx:8,} of {len(filenames):8,} ({datetime.datetime.now()})')
    content = get_content(filename)
    for token in nlp.tokenizer(content):
        if not token.is_stop:
            t = token.text.replace('\n','')
            freq[t]+=1
            if freq[t]==2:
                writer.write(f'{t}\n')
writer.close()
print(len(freq))

Processed        0 of  192,433 (2022-02-02 18:03:28.490711)
Processed   10,000 of  192,433 (2022-02-02 18:04:09.802195)
Processed   20,000 of  192,433 (2022-02-02 18:04:44.722166)
Processed   30,000 of  192,433 (2022-02-02 18:05:17.298169)
Processed   40,000 of  192,433 (2022-02-02 18:05:48.787908)
Processed   50,000 of  192,433 (2022-02-02 18:07:13.301679)
Processed   60,000 of  192,433 (2022-02-02 18:08:38.424454)
Processed   70,000 of  192,433 (2022-02-02 18:10:02.034738)
Processed   80,000 of  192,433 (2022-02-02 18:11:22.875957)
Processed   90,000 of  192,433 (2022-02-02 18:12:42.589241)
Processed  100,000 of  192,433 (2022-02-02 18:14:00.350605)
Processed  110,000 of  192,433 (2022-02-02 18:15:20.642944)
Processed  120,000 of  192,433 (2022-02-02 18:16:39.921008)
Processed  130,000 of  192,433 (2022-02-02 18:17:57.841917)
Processed  140,000 of  192,433 (2022-02-02 18:19:14.589893)
Processed  150,000 of  192,433 (2022-02-02 18:20:29.239788)
Processed  160,000 of  192,433 (2022-02-

In [52]:
!cat vocab.txt | wc -l

2171420


In [40]:
import string
import numpy as np
freq = {}

def filter_token(word):
    word = word.replace('\n','')
    word = word.lower()
    if not word.isnumeric() and len(''.join([char for char in word if not char in string.punctuation])): # LEN == QUE??
        return word
    else:
        return None
    

for token in open('vocab.txt', 'r').read().splitlines():
    token = filter_token(token)
    if not token is None:
        freq[token]=0

for idx,filename in enumerate(filenames):
    if (idx) % 10000==0:
        print(f'Processed {idx:8,} of {len(filenames):8,} ({datetime.datetime.now()})')
        
    content = get_content(filename)
    for token in nlp.tokenizer(content):
        t = filter_token(token.text)
        if t in freq:
            freq[t]+=1

print(f'{len(freq):,}')
print(np.random.choice([word for word in freq],size=20,replace=False))

Processed        0 of  192,433 (2022-02-02 19:52:17.316400)
Processed   10,000 of  192,433 (2022-02-02 19:52:56.695617)
Processed   20,000 of  192,433 (2022-02-02 19:53:36.739117)
Processed   30,000 of  192,433 (2022-02-02 19:54:17.773373)
Processed   40,000 of  192,433 (2022-02-02 19:54:56.308136)
Processed   50,000 of  192,433 (2022-02-02 19:56:02.310448)
Processed   60,000 of  192,433 (2022-02-02 19:57:14.398479)
Processed   70,000 of  192,433 (2022-02-02 19:58:26.415487)
Processed   80,000 of  192,433 (2022-02-02 19:59:40.181994)
Processed   90,000 of  192,433 (2022-02-02 20:00:53.659325)
Processed  100,000 of  192,433 (2022-02-02 20:02:07.289399)
Processed  110,000 of  192,433 (2022-02-02 20:03:21.241653)
Processed  120,000 of  192,433 (2022-02-02 20:04:35.424931)
Processed  130,000 of  192,433 (2022-02-02 20:05:48.341121)
Processed  140,000 of  192,433 (2022-02-02 20:07:00.799116)
Processed  150,000 of  192,433 (2022-02-02 20:08:13.155880)
Processed  160,000 of  192,433 (2022-02-

In [41]:
for i in list(range(20))+[192, 1920]:
    print(f'umbral: {i:3} - Cantidad de documentos en vocabulario: {len([word for word in freq if freq[word]>i]):10,} (freq[word]>{i})')

umbral:   0 - Cantidad de documentos en vocabulario:  1,486,315 (freq[word]>0)
umbral:   1 - Cantidad de documentos en vocabulario:  1,486,315 (freq[word]>1)
umbral:   2 - Cantidad de documentos en vocabulario:  1,002,528 (freq[word]>2)
umbral:   3 - Cantidad de documentos en vocabulario:    760,010 (freq[word]>3)
umbral:   4 - Cantidad de documentos en vocabulario:    619,974 (freq[word]>4)
umbral:   5 - Cantidad de documentos en vocabulario:    529,571 (freq[word]>5)
umbral:   6 - Cantidad de documentos en vocabulario:    466,150 (freq[word]>6)
umbral:   7 - Cantidad de documentos en vocabulario:    419,431 (freq[word]>7)
umbral:   8 - Cantidad de documentos en vocabulario:    383,317 (freq[word]>8)
umbral:   9 - Cantidad de documentos en vocabulario:    354,064 (freq[word]>9)
umbral:  10 - Cantidad de documentos en vocabulario:    330,101 (freq[word]>10)
umbral:  11 - Cantidad de documentos en vocabulario:    310,310 (freq[word]>11)
umbral:  12 - Cantidad de documentos en vocabulari

True

In [53]:
word_frequency_list = [(word, freq2[word]) for word in freq if not word.strip()=='' and len(word)>=3]
word_frequency_list = sorted(word_frequency_list, key=lambda x: x[1],reverse=True)
new_vocab = [word for word,_ in word_frequency_list[:50000]]
new_vocab[:50]

['toronto',
 'new',
 'said',
 'room',
 'canada',
 'home',
 'time',
 'good',
 'movie',
 'man',
 'year',
 'people',
 'day',
 'world',
 'years',
 'tho',
 'city',
 'work',
 'street',
 'wanted',
 'business',
 'york',
 'house',
 'phone',
 'news',
 'canadian',
 'school',
 'large',
 'box',
 'ing',
 'west',
 'ontario',
 'ont',
 'family',
 'men',
 'ltd',
 'experience',
 'old',
 'globe',
 'country',
 'government',
 'apply',
 'war',
 'sale',
 'life',
 'north',
 'like',
 'john',
 'second',
 'office']

In [55]:
f'{len(new_vocab):,}'

'50,000'

In [57]:
writer = open('vocab/vocab_filtered.txt', 'w')
writer.write('\n'.join(new_vocab))
writer.close()